In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 150

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
plt.style.use('dark_background')

### 1.2 Bibliotecas para MLP

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

from tensorflow import keras

### 1.3 Bibliotecas dos sistemas caóticos

In [13]:
import sys 
sys.path.insert(0, '../../../../scripts')

import timeseries as times
import mlp_model as mlp
import logisticmap as logmap

## 2. Gerando a série temporal do mapa logístico

Os valores para os parâmetros das equações serão os "básicos" apresentados no *paper* original. Utilizaremos  $r = 3.86$, o que faz com que o sistema esteja em caos.

In [4]:
x_inicial = 0.5
n_inicial = 0
n_iteracoes = 5000
r=3.86
estados = np.array([x_inicial, n_inicial])

In [5]:
mapa = logmap.MapaLogistico(estado_inicial=estados, r=r)

In [6]:
for n in range(0, n_iteracoes):
    mapa.iterar()
    estados = np.vstack((estados, mapa.ler_estado()))

In [7]:
x = estados[:, 0]
n = estados[:, 1]

In [8]:
fig, ax = plt.subplots()
ax.plot(n, x, color='Crimson')

ax.set_title("$100$ iterações iniciais da série temporal do Mapa Logístico\n para $r =$ " + str(r) + " com $x[0] =$ " + str(x[0]))
ax.set_ylabel('$x[n]$')
ax.set_xlabel('$n$')
ax.set_xlim(0,100)
    
ax.grid(True)
sns.despine()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.1 Dividindo em um conjunto de treinamento e de teste, para K = 4 e L = 3

In [9]:
K = 4
L = 3
tam_teste = 0.15
tam_val = 0.1

In [10]:
serie_temporal = times.SerieTemporal(x, K=K, L=L)

In [11]:
serie_temporal.criar_matrizes()

In [12]:
X_treino, X_val, X_teste, y_treino, y_val, y_teste = serie_temporal.dividir_treino_teste_validacao(tam_teste, tam_val)

## 3. Usando o modelo com a classe

In [27]:
batch_normalization = 'OFF'
activation = 'tanh'
init_mode = 'glorot_uniform'
n_neurons = 10
n_hidden_layers = 3

optimizer = 'Nadam'
learning_rate = 0.003
loss = "mean_squared_error"

batch_size = 2

In [28]:
MLP = mlp.ModeloMLP(input_size = K, name="MLP-logistic")

In [29]:
MLP.construir_mlp(batch_normalization, activation,
                  init_mode, n_neurons,
                  n_hidden_layers)

TypeError: construir_mlp() takes from 0 to 5 positional arguments but 6 were given